In [ ]:
def calculate_weights(points_dict):
    """
    Calculate weights for names based on points with minimum and maximum constraints.
    Ensures weights sum to exactly 100% after rounding.
    
    Args:
        points_dict (dict): Dictionary with names as keys and points as values
        
    Returns:
        dict: Dictionary with names and their calculated weights as percentages
    """
    n = len(points_dict)
    min_weight = 100 / (2 * n)  # Minimum weight percentage
    max_weight = 15.0  # Maximum weight percentage
    
    # Calculate initial weights
    total_points = sum(points_dict.values())
    initial_weights = {name: (points/total_points) * 100 
                      for name, points in points_dict.items()}
    
    # Identify names below minimum threshold
    below_min = {name: weight for name, weight in initial_weights.items() 
                if weight < min_weight}
    above_min = {name: weight for name, weight in initial_weights.items() 
                if weight >= min_weight}
    
    # Set minimum weights for those below threshold
    final_weights = {}
    for name in below_min:
        final_weights[name] = min_weight
    
    # Calculate remaining weight to distribute
    total_min_weight = len(below_min) * min_weight
    remaining_weight = 100 - total_min_weight
    
    # Redistribute remaining weight proportionally while respecting max_weight
    if above_min:
        total_above_min = sum(above_min.values())
        
        # First pass: distribute remaining weight proportionally
        for name, weight in above_min.items():
            scaled_weight = (weight / total_above_min) * remaining_weight
            if scaled_weight > max_weight:
                final_weights[name] = max_weight
            else:
                final_weights[name] = scaled_weight
        
        # Check if we need to redistribute excess weight
        total_allocated = sum(final_weights.values())
        if total_allocated < 100:
            excess = 100 - total_allocated
            non_max_weights = {n: w for n, w in final_weights.items() 
                             if w < max_weight and n not in below_min}
            
            if non_max_weights:
                total_non_max = sum(non_max_weights.values())
                for name in non_max_weights:
                    additional = (non_max_weights[name] / total_non_max) * excess
                    final_weights[name] += additional
    
    # Round to 2 decimal places while ensuring sum is exactly 100
    rounded_weights = {}
    remaining = 100.0
    sorted_items = sorted(final_weights.items(), key=lambda x: x[1], reverse=True)
    
    # Round all but the last weight
    for name, weight in sorted_items[:-1]:
        rounded_weight = round(weight, 2)
        rounded_weights[name] = rounded_weight
        remaining -= rounded_weight
    
    # Assign the remaining weight to the last item to ensure sum is exactly 100
    last_name, _ = sorted_items[-1]
    rounded_weights[last_name] = round(remaining, 2)
    
    return rounded_weights

# Example usage
points = {
    'shop': 96, 'amzn': 96, 'axp': 96, 'cat': 96, 'pypl': 96, 'c': 96,
    'bac': 96, 'bb': 96, 'usb': 9, 'acn': 0, 'txn': 0, 'aig': 0
}

weights = calculate_weights(points)
# Print sorted by weight
total = 0
for name, weight in sorted(weights.items(), key=lambda x: x[1], reverse=True):
    print(f"{name}: {weight}%")
    total += weight
print(f"\nTotal: {total}%")